In [2]:
# !pip install pandas==1.3.4
# !pip install plotly==5.3.1
!pip install kaleido==0.2.1

     |████████████████████████████████| 79.9 MB 1.2 MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# import LSTNet (https://github.com/flaviagiammarino/lstnet-tensorflow)
# !ls /content/gdrive/MyDrive/Masterarbeit/Code/lstnet-tensorflow/lstnet_tensorflow/*.py
import sys
sys.path.append('/content/gdrive/MyDrive/Masterarbeit/Code/lstnet-tensorflow/')
import lstnet_tensorflow

In [8]:
import numpy as np
data1 = np.load('/content/gdrive/MyDrive/Masterarbeit/Code/two-hearts/sample-data/0001.npy')
data2 = np.load('/content/gdrive/MyDrive/Masterarbeit/Code/two-hearts/sample-data/0016.npy')

In [5]:
print(data1)

[-0.03374548 -0.00912922  0.01347736 ...  0.02617211  0.03393547
  0.04377749]


In [18]:
import numpy as np
from lstnet_tensorflow.model import LSTNet

# Generate two time series
# N = 500
# t = np.linspace(0, 1, N)
# e = np.random.multivariate_normal(mean=[0, 0], cov=[[1, 0.25], [0.25, 1]], size=N)
# a = 40 + 30 * t + 20 * np.cos(2 * np.pi * (10 * t - 0.5)) + e[:, 0]
# b = 80 - 20 * t + 10 * np.sin(2 * np.pi * (20 * t - 0.5)) + e[:, 1]
a = data1[:10000]
b = data2[:10000]
y = np.hstack([a.reshape(- 1, 1), b.reshape(- 1, 1)])
print(y)

[[-0.03374548 -0.01782184]
 [-0.00912922 -0.01752107]
 [ 0.01347736 -0.01598894]
 ...
 [ 0.4395004  -0.03511794]
 [ 0.33345823 -0.03743859]
 [ 0.02494192 -0.03732312]]


In [21]:

# Fit the model 
# TODO: set back to default
model = LSTNet(
    y=y,
    forecast_period=100,
    lookback_period=300,
    kernel_size=100,
    filters=4,
    gru_units=100,
    skip_gru_units=0,
    skip=0,
    lags=100,
)

model.fit(
    loss='mae',
    learning_rate=0.01,
    batch_size=64,
    epochs=32,
    verbose=1
)

Epoch 1/32
152/152 [==============================] - 63s 309ms/step - loss: 0.0555
Epoch 2/32
152/152 [==============================] - 47s 312ms/step - loss: 0.0147
Epoch 3/32
152/152 [==============================] - 47s 309ms/step - loss: 0.0124
Epoch 4/32
152/152 [==============================] - 47s 312ms/step - loss: 0.0119
Epoch 5/32
152/152 [==============================] - 47s 311ms/step - loss: 0.0132
Epoch 6/32
152/152 [==============================] - 48s 312ms/step - loss: 0.0129
Epoch 7/32
152/152 [==============================] - 48s 314ms/step - loss: 0.0124
Epoch 8/32
152/152 [==============================] - 48s 315ms/step - loss: 0.0131
Epoch 9/32
152/152 [==============================] - 47s 312ms/step - loss: 0.0127
Epoch 10/32
152/152 [==============================] - 47s 312ms/step - loss: 0.0123
Epoch 11/32
152/152 [==============================] - 47s 311ms/step - loss: 0.0121
Epoch 12/32
152/152 [==============================] - 47s 311ms/step - lo

In [22]:
# Plot the in-sample predictions
predictions = model.predict(index=400)
fig = model.plot_predictions()
#fig.write_image('predictions.png', width=750, height=650)
fig.show()

In [23]:
# Plot the out-of-sample forecasts
forecasts = model.forecast()
fig = model.plot_forecasts()
#fig.write_image('forecasts.png', width=750, height=650)
fig